In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
#export
import sys
sys.path.append("..")
from faigen.data import sequence 
from faigen.data.sequence import regex_filter, count_filter, Dna2VecDataBunch,Dna2VecList, seq_record
from functools import partial
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import manifold,neighbors
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
import plotly.plotly as py
import plotly.graph_objs as go
from fastai import *
from fastai.data_block import *
from fastai.basic_train import *
from fastai.layers import *
from fastai.metrics import *
from fastai.text import *
from gensim.models import Word2Vec
import torch 
import torch.nn as nn
import torch.nn.functional as F
import gc 
from itertools import islice
from tqdm import tqdm


In [4]:
#export
print("Loading embedding")
word_vectors = Word2Vec.load_word2vec_format('/data/genomes/embeddings/dna2vec-20190612-1611-k10to10-100d-10c-4870Mbp-sliding-kPR.w2v') 

Loading embedding


In [5]:
#export
# DB="/data/genomes/GenSeq_fastas"
# DB='/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes'
# DB="/home/serge/database/data/genomes/ncbi-genomes-2019-04-07/Bacillus"
DB="/home/serge/database/data/genomes/bacillus/ncbi-genomes-2019-06-25"
data, X, dfx = None,None,None
bunch=None
learner=None
gc.collect()

14495

In [6]:
filters=[partial(regex_filter, rx="plasmid", keep=False)]
data = sequence.Dna2VecList.from_folder(DB,filters=filters,n_cpus=7,emb=word_vectors,recurse=True)
sequence.GSFileProcessor().process(data)


100%|██████████| 3964/3964 [05:12<00:00, 14.89it/s]


In [58]:
dfseq = pd.DataFrame.from_dict({"seq": list(map(str, data.items)),
                                "description": data.descriptions, 
                                "file":data.files,
                                "id":data.ids, 
                                "name":data.names})

In [60]:
dfseq.to_pickle("/home/serge/database/data/genomes/bacillus/ncbi-genomes-2019-06-25/all_sequences-no-plasmid.pkl")

In [9]:
def k_mers(sequence, k):
    it = iter(sequence)
    result = tuple(islice(it, k))
    if len(result) == k:
        yield "".join(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield "".join(result)

In [7]:
data.items[0]

Seq('ATTTCCCATGAAATAGGTTCGGTTCTGTTAGTAAAAAATTCGAAATATAGTAAG...NNN', SingleLetterAlphabet())

In [13]:
mers = np.asarray([word_vectors[x] for x in k_mers(str(data.items[0]), 10) if set(x) == set('ATGC')])

In [17]:
mers.mean(axis=0).shape

(100,)

In [44]:
class Vectorizer:
    def __init__(self,texts=None,  ngram=10, skip=0, n_cpus=7, chunksize=1000):
        self.texts, self.ngram, self.skip, self.n_cpus, self.chunksize = texts, ngram, skip, n_cpus, chunksize
                    
    def vectorizer(self, t):
        if self.ngram == 1:
            toks = list(t)
            if self.skip > 0:
                toks = toks[::2] if self.skip == 1 else toks[::self.skip]
        else:
            toks = [t[i:i + self.ngram] for i in range(0, len(t), self.ngram + self.skip) if i+self.ngram < len(t)] 
        res = np.asarray(word_vectors[filter(lambda x: set(x) == set("ACGT"), toks)]).mean(axis=0)
        toks=None
        return res

    def _process_all_1(self, texts):
        return [self.vectorizer(str(t)) for t in texts]

    def process_all(self, texts):
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            res = sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])
        return res
        
    def vectorize(self,texts=None):
        texts = self.texts if self.texts is not None else texts
        vectors = []
        chunks = len(texts) // self.chunksize + 1
        for i in tqdm(range(chunks)):
            advance = min((len(texts) - i * self.chunksize), self.chunksize)
            vectors += self.process_all(texts[i:i + advance])
        return vectors

In [3]:
sum(list([1]),[])

TypeError: can only concatenate list (not "int") to list

In [6]:
ProcessPoolExecutor??

Object `ProcessPoolExecutor` not found.


In [5]:
res= partition_by_cores(data.items, 7)

NameError: name 'partition_by_cores' is not defined

In [51]:
len(res)

7

In [45]:
vectors = Vectorizer().vectorize(data.items)








  0%|          | 0/448 [00:00<?, ?it/s]

TypeError: zip argument #2 must support iteration

In [26]:
vectors = []
for sequence in tqdm(data.items):
    mers = np.asarray([word_vectors[x] for x in k_mers(sequence, 10) if set(x) == set('ATGC')])
    vectors.append(mers.mean(axis=0))
    


  0%|          | 0/447112 [00:00<?, ?it/s]

KeyboardInterrupt: 